In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load data
df_gdp_mainlandchina = pd.read_csv('raw_data/data-gdp-mainlandchina-2019.csv')  
df_gdp_usa = pd.read_csv('raw_data/data-gdp-usa-2019.csv')  
df_pop_china = pd.read_csv('raw_data/data-pop-china-2017.csv')  
df_pop_usa = pd.read_csv('raw_data/data-pop-usa-2019.csv')  
df_WEO = pd.read_csv('raw_data/WEO_Data.csv')  
df_region_china = pd.read_csv('raw_data/data-economyregion-china.csv')  

# China

In [3]:
# Merge dataframe for China
df_china = pd.DataFrame.merge(df_gdp_mainlandchina,df_pop_china,how='right',left_on='Provinces',right_on='Administrative Division')
df_china = pd.DataFrame.merge(df_china,df_region_china,how='left',left_on='Administrative Division',right_on='Provinces')

# Don't show Unnamed columns
df_china = df_china.loc[:, ~df_china.columns.str.contains('^Unnamed')] 

# Drop column
df_china = df_china.drop(['Provinces_x'], axis=1)
df_china = df_china.drop(['Provinces_y'], axis=1)

# Add column
df_china['Country'] = 'China'
df_china['Year'] = '2019'

df_china.tail()

,Nominal GDP (Billion),Administrative Division,Population,Region,Economic Zone #,Economic Zone,经济区,Characteristic,Country,Year
29,43.0,Qinghai,5980000,West,10,Qinghai-Tibet Plateau,青藏高原经济区,"Natural gas, salt lake resources, non-ferrous ...",China,2019
30,25.0,Tibet,3370000,West,10,Qinghai-Tibet Plateau,青藏高原经济区,"Natural gas, salt lake resources, non-ferrous ...",China,2019
31,NaN,Hong Kong,7335384,Hong Kong,11,Hong Kong,香港特别行政区,"Trade and logistics industry (21.2%), financia...",China,2019
32,NaN,Macau,644900,Macau,12,Macau,澳门港特别行政区,Tourism and apparel industry (41%),China,2019
33,NaN,Taiwan,23562318,Taiwan,13,Taiwan,台澎金马个别关税领域,Agricultural products and OEM,China,2019


In [4]:
# Get GDP value for Hong Kong, Macau and Taiwan
df_hongkong = df_WEO[df_WEO['Country'].str.contains('hong kong', case=False, na=False)]
hongkong_gdp = df_hongkong.loc[df_hongkong['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

df_macau = df_WEO[df_WEO['Country'].str.contains('maca', case=False, na=False)]
macau_gdp = df_macau.loc[df_macau['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

df_taiwan = df_WEO[df_WEO['Country'].str.contains('taiwan', case=False, na=False)]
taiwan_gdp = df_taiwan.loc[df_taiwan['Subject Descriptor'] == 'Gross domestic product, current prices', '2019'].values[0]

print('Hong Kong GDP: ',hongkong_gdp)
print('Macau GDP: ',macau_gdp)
print('Taiwan GDP: ',taiwan_gdp)

Hong Kong GDP:  372.989
Macau GDP:  55.136
Taiwan GDP:  586.104


In [5]:
# Assign value
## to Hong Kong
hongkong_index = df_china.loc[df_china['Administrative Division'] == 'Hong Kong'].index[0]
df_china.at[hongkong_index, 'Nominal GDP (Billion)'] = hongkong_gdp

## to Macau
macau_index = df_china.loc[df_china['Administrative Division'] == 'Macau'].index[0]
df_china.at[macau_index, 'Nominal GDP (Billion)'] = macau_gdp

## to Taiwan 
taiwan_index = df_china.loc[df_china['Administrative Division'] == 'Taiwan'].index[0]
df_china.at[taiwan_index, 'Nominal GDP (Billion)'] = taiwan_gdp

# Calculate GDP per Capita
df_china['GDP per Capita'] = (df_china['Nominal GDP (Billion)'] / df_china['Population'])*1000000000

# Re-order Columns
df_china = df_china[['Year','Country','Region','Economic Zone #','Economic Zone','Administrative Division','Nominal GDP (Billion)','Population','GDP per Capita','Characteristic']]

# Set 0 decimal places
df_china['GDP per Capita'] = df_china['GDP per Capita'].round(0)

df_china

,Year,Country,Region,Economic Zone #,Economic Zone,Administrative Division,Nominal GDP (Billion),Population,GDP per Capita,Characteristic
0,2019,China,East,4,Southeast Coast,Guangdong,1561.000,111690000,13976.0,"Export-oriented, digesting foreign advanced te..."
1,2019,China,East,3,East Coast,Jiangsu,1444.000,80290000,17985.0,Multifunctional manufacturing: Light industria...
2,2019,China,East,2,North Coast,Shandong,1030.000,100060000,10294.0,High-tech R & D and manufacturing
3,2019,China,East,3,East Coast,Zhejiang,904.000,56570000,15980.0,Multifunctional manufacturing: Light industria...
4,2019,China,Middle,5,Yellow River Upper and Middle Reaches,Henan,787.000,95590000,8233.0,"Coal mining and coal deep processing, natural ..."
5,2019,China,West,6,Yangtze River Upper and Middle Reaches,Sichuan,676.000,83020000,8143.0,"Rice and cotton, steel and non-ferrous metallu..."
6,2019,China,Middle,6,Yangtze River Upper and Middle Reaches,Hubei,664.000,59020000,11250.0,"Rice and cotton, steel and non-ferrous metallu..."
7,2019,China,East,4,Southeast Coast,Fujian,615.000,39110000,15725.0,"Export-oriented, digesting foreign advanced te..."
8,2019,China,Middle,6,Yangtze River Upper and Middle Reaches,Hunan,576.000,68600000,8397.0,"Rice and cotton, steel and non-ferrous metallu..."
9,2019,China,East,3,East Coast,Shanghai,553.000,24180000,22870.0,Multifunctional manufacturing: Light industria...


# USA

In [6]:
df_gdp_usa.head()

,State,Region,Nominal GDP (Billion)
0,Connecticut,New England,288.99
1,Maine,New England,68.44
2,Massachusetts,New England,604.21
3,New Hampshire,New England,89.84
4,Rhode Island,New England,64.44


In [7]:
df_pop_usa.head()

,State,Population
0,California,39512223
1,Texas,28995881
2,Florida,21477737
3,New York,19453561
4,Pennsylvania,12801989


In [8]:
# Strip leading and trailng white space in State column
df_gdp_usa['State'] = df_gdp_usa['State'].str.strip()
df_pop_usa['State'] = df_pop_usa['State'].str.strip()

# Merge dataframe for China
df_usa = pd.DataFrame.merge(df_gdp_usa,df_pop_usa,how='left',on='State')

# Add column
df_usa['Country'] = 'United States'
df_usa['Year'] = '2019'

# Rename column
df_usa.rename(columns={'State':'Administrative Division'}, inplace=True)

# Re-order Columns
df_usa = df_usa[['Year','Country','Administrative Division','Nominal GDP (Billion)','Population']]

# Calculate GDP per Capita
df_usa['GDP per Capita'] = (df_usa['Nominal GDP (Billion)'] / df_usa['Population'])*1000000000

# Set 0 decimal places
df_usa['GDP per Capita'] = df_usa['GDP per Capita'].round(0)

df_usa

,Year,Country,Administrative Division,Nominal GDP (Billion),Population,GDP per Capita
0,2019,United States,Connecticut,288.99,3565287,81057.0
1,2019,United States,Maine,68.44,1344212,50915.0
2,2019,United States,Massachusetts,604.21,6949503,86943.0
3,2019,United States,New Hampshire,89.84,1359711,66073.0
4,2019,United States,Rhode Island,64.44,1059361,60829.0
5,2019,United States,Vermont,35.27,623989,56523.0
6,2019,United States,Delaware,76.41,973764,78469.0
7,2019,United States,District of Columbia,148.23,705749,210032.0
8,2019,United States,Maryland,434.31,6045680,71838.0
9,2019,United States,New Jersey,652.41,8882190,73451.0


# Concat

In [9]:
# Concat data frame
clean_df = pd.concat([df_china, df_usa])

# Reset index
clean_df.reset_index(drop = True, inplace = True)

clean_df.head()

,Administrative Division,Characteristic,Country,Economic Zone,Economic Zone #,GDP per Capita,Nominal GDP (Billion),Population,Region,Year
0,Guangdong,"Export-oriented, digesting foreign advanced te...",China,Southeast Coast,4.0,13976.0,1561.0,111690000,East,2019
1,Jiangsu,Multifunctional manufacturing: Light industria...,China,East Coast,3.0,17985.0,1444.0,80290000,East,2019
2,Shandong,High-tech R & D and manufacturing,China,North Coast,2.0,10294.0,1030.0,100060000,East,2019
3,Zhejiang,Multifunctional manufacturing: Light industria...,China,East Coast,3.0,15980.0,904.0,56570000,East,2019
4,Henan,"Coal mining and coal deep processing, natural ...",China,Yellow River Upper and Middle Reaches,5.0,8233.0,787.0,95590000,Middle,2019


In [10]:
clean_df.to_csv('clean_data/clean_china_and_usa.csv')